In [1]:
import ot
import ot.plot
import ot.bregman
import ot.lp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from IPython.display import display, clear_output

In [2]:
n_values = 500
n_bids = 500

values = np.linspace(0,1,n_values)
bids = np.linspace(0,1,n_bids)
my_density = np.ones((n_bids,))/n_bids
opponent_density = np.ones((n_bids,))/n_bids
value_density = np.ones((n_values,))/n_values

In [3]:
def cumulative_bid_distribution(bid_density):
    return np.cumsum(bid_density) - 0.5 * bid_density

def utility_matrix(opponent_density, values, bids):
    my_winning_probability = cumulative_bid_distribution(opponent_density)
    M = (values[:,None] - bids) * my_winning_probability
    return M

def compute_gradient(value_density, my_density, utility_matrix):
    M = -utility_matrix
    plan, log = ot.emd(value_density, my_density, M, log = True)
    return -log['v'], plan

def vec2simplexV1(vecX, l=1.):
    m = vecX.size
    vecS = np.sort(vecX)[::-1]
    vecC = np.cumsum(vecS) - l
    vecH = vecS - vecC / (np.arange(m) + 1)
    r = np.max(np.where(vecH>0)[0])
    t = vecC[r] / (r + 1)
    return np.maximum(0, vecX - t)



In [ ]:
#reg = 1e-2
#plan, log = ot.bregman.sinkhorn(value_density, my_density, M, reg, log = True)
#plan, log = ot.emd(value_density, my_density, M, log = True)
#ot.plot.plot1D_mat(value_density, my_density, plan, 'yolo')

In [ ]:
density_1 = vec2simplexV1(np.random.uniform(0.,1.,(n_bids,)))
density_2 = vec2simplexV1(np.random.uniform(0.,1.,(n_bids,)))
step_size = .01
n_iterations = 3000

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

for i in tqdm(range(n_iterations)):
    utility_1 = utility_matrix(density_2, values, bids)
    utility_2 = utility_matrix(density_1, values, bids)
    gradient_1, _ = compute_gradient(value_density, density_1, utility_1)
    gradient_2, _ = compute_gradient(value_density, density_2, utility_2)
    density_1 = vec2simplexV1(density_1 + 10.*(1./(i+1.))*gradient_1)
    density_2 = vec2simplexV1(density_2 + 10*(1./(i+1.))*gradient_2)

    if i % 50 == 0:
        clear_output(wait=True)
        ax.cla()
        ax.plot(density_1)
        display(fig)


In [ ]:
equilibrium_strategy = np.zeros((n_bids))
for i in range(int(n_bids/2)):
    equilibrium_strategy[i] = 1.
equilibrium_strategy = vec2simplexV1(equilibrium_strategy)

num_tests = 1000

distance_inner_pairs = []
for _ in tqdm(range(num_tests)):
    density_1 = vec2simplexV1(np.random.uniform(0.,1.,(n_bids,)))
    density_2 = vec2simplexV1(np.random.uniform(0.,1.,(n_bids,)))
    utility_1 = utility_matrix(density_2, values, bids)
    gradient_1, _ = compute_gradient(value_density, density_1, utility_1)
    x = np.inner(gradient_1, equilibrium_strategy - density_1)
    distance_inner_pairs.append((np.linalg.norm(equilibrium_strategy - density_1), x ))
    #if x < 0.:
    #    print("Au weia")
    #    print(x)
    #    fig, ax = plt.subplots(1,2)
    #    ax[0].plot(density_1)
    #    ax[1].plot(density_2)
    #    break

plt.scatter([x for (x,_) in distance_inner_pairs],[y for (_,y) in distance_inner_pairs])


In [4]:
equilibrium_strategy = np.zeros((n_bids))
for i in range(int(n_bids/2)):
    equilibrium_strategy[i] = 1.
equilibrium_strategy = vec2simplexV1(equilibrium_strategy)

num_tests = 1000

distance_inner_pairs = []
for _ in tqdm(range(num_tests)):
    i1 = np.random.randint(0,n_bids)
    i2 = np.random.randint(0,n_bids)
    density_1 = np.zeros((n_bids,))
    density_1[:i1] = 1
    density_1 = vec2simplexV1(density_1)
    density_2 = np.zeros((n_bids,))
    density_2[:i2] = 1
    density_2 = vec2simplexV1(density_2)
    utility_1 = utility_matrix(density_2, values, bids)
    gradient_1, _ = compute_gradient(value_density, density_1, utility_1)
    x = np.inner(gradient_1, equilibrium_strategy - density_1)
    distance_inner_pairs.append((np.linalg.norm(equilibrium_strategy - density_1), x ))
    if x < -0.1:
        print(i1)
        print(i2)


plt.scatter([x for (x,_) in distance_inner_pairs],[y for (_,y) in distance_inner_pairs])


  1%|          | 8/1000 [00:00<00:28, 35.36it/s]

102
37


  7%|▋         | 73/1000 [00:03<00:34, 27.11it/s]

95
45


 10%|█         | 105/1000 [00:04<00:28, 31.84it/s]

...

 53%|█████▎    | 533/1000 [00:20<00:13, 35.40it/s]

85
21
115
32


 68%|██████▊   | 676/1000 [00:25<00:12, 26.65it/s]


111
31


KeyboardInterrupt: 